<a href="https://colab.research.google.com/github/chathasphere/chathasphere.github.io/blob/main/teaching/306_materials/003_lab7_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 7: Missing Values and Cleaning Messy Data

## March 15th, 2022

In [1]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
(stocks <- tibble(
  Year   = c(2015, 2015, 2015, 2015, 2016, 2016, 2016),
  Qtr    = c(   1,    2,    3,    4,    2,    3,    4),
  Return = c(1.88, 0.59, 0.35,   NA, 0.92, 0.17, 2.66)
))

Year,Qtr,Return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,NA
2016,2,0.92
2016,3,0.17
2016,4,2.66


# 1. Missing Values
Missing values can be:
1. *Explicit* (marked as NA in our data)
1. *Implicit* (not present in the data)

In this example we have one explicitly missing value for the 4th quarter of 2015. 

Are there any other missing values? Yes: we do not have an observation for the first quarter of 2016.

## Handling Missing Data

`complete`: Turns implicit missing values into explicit missing values.

Specify a list of columns (column separated) to generate every possible combination. Missing combinations will initialize columns with NA.

In [ ]:
stocks %>% complete(Year, Qtr)

Year,Qtr,Return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,NA
2016,1,NA
2016,2,0.92
2016,3,0.17
2016,4,2.66


`fill`: Fills missing values in selected columns. Defaults to using the previous entry.

In [ ]:
stocks %>% complete(Year, Qtr) %>% fill(Return)

Year,Qtr,Return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,0.35
2016,1,0.35
2016,2,0.92
2016,3,0.17
2016,4,2.66


In [ ]:
stocks %>% complete(Year, Qtr) %>% fill(Return, .direction="up")

Year,Qtr,Return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,0.92
2016,1,0.92
2016,2,0.92
2016,3,0.17
2016,4,2.66


The missing values also become explicit if we widen the tibble.

In [ ]:
stocks_wide = stocks %>% pivot_wider(names_from = Year, values_from = Return)

stocks_wide

Qtr,2015,2016
<dbl>,<dbl>,<dbl>
1,1.88,NA
2,0.59,0.92
3,0.35,0.17
4,NA,2.66


`pivot_longer` will keep all these explicitly missing values by default.

In [ ]:
stocks_wide %>% pivot_longer(cols = `2015`:`2016`, names_to = 'Year') %>%
arrange(Year)

Qtr,Year,value
<dbl>,<chr>,<dbl>
1,2015,1.88
2,2015,0.59
3,2015,0.35
4,2015,NA
1,2016,NA
2,2016,0.92
3,2016,0.17
4,2016,2.66


# 2. Cleaning messy data

In [13]:
datacamp_url = "https://assets.datacamp.com/production/repositories/34/datasets/b3c1036d9a60a9dfe0f99051d2474a54f76055ea/weather.rds"
weather = readRDS(url(datacamp_url))

In [ ]:
weather %>% glimpse

Rows: 286
Columns: 35
$ X       <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,…
$ year    <int> 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 20…
$ month   <int> 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12…
$ measure <chr> "Max.TemperatureF", "Mean.TemperatureF", "Min.TemperatureF", "…
$ X1      <chr> "64", "52", "39", "46", "40", "26", "74", "63", "52", "30.45",…
$ X2      <chr> "42", "38", "33", "40", "27", "17", "92", "72", "51", "30.71",…
$ X3      <chr> "51", "44", "37", "49", "42", "24", "100", "79", "57", "30.4",…
$ X4      <chr> "43", "37", "30", "24", "21", "13", "69", "54", "39", "30.56",…
$ X5      <chr> "42", "34", "26", "37", "25", "12", "85", "66", "47", "30.68",…
$ X6      <chr> "45", "42", "38", "45", "40", "36", "100", "93", "85", "30.42"…
$ X7      <chr> "38", "30", "21", "36", "20", "-3", "92", "61", "29", "30.69",…
$ X8      <chr> "29", "24", "18", "28", "16", "3", "92", "70", "47", "30.77", …
$ X9      <chr> "4

In [ ]:
weather %>% head

The first column lists row number, so let's ignore it.

In [14]:
weather <- weather %>% select(-X)

It looks like the values for the weather measurements (column 3) for each day of the month are stored in the columns `X1` to `X31`. From a **tidy data** perspective, the data set is messy because:
* Values are given as column names (`X1` to `X31`)
* variable names are represented as values (column 3 - `measure`)

We can correct it by using `pivot_longer`.

In [15]:
tidy_weather <- weather %>% 
  pivot_longer(cols = `X1`:`X31`, names_to = 'day', values_to = "value") %>%
  select(year, month, day, everything())

head(tidy_weather)

year,month,day,measure,value
<int>,<int>,<chr>,<chr>,<chr>
2014,12,X1,Max.TemperatureF,64
2014,12,X2,Max.TemperatureF,42
2014,12,X3,Max.TemperatureF,51
2014,12,X4,Max.TemperatureF,43
2014,12,X5,Max.TemperatureF,42
2014,12,X6,Max.TemperatureF,45


The values in the column `measure` of the weather dataset should be variables.

In [16]:
tidy_weather <- tidy_weather %>% 
  pivot_wider(names_from = measure, values_from = value)

head(tidy_weather)

year,month,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,⋯,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2014,12,X1,64,52,39,46,40,26,74,⋯,10,10,10,22,13,29,0.01,6,Rain,268
2014,12,X2,42,38,33,40,27,17,92,⋯,10,8,2,24,15,29,0.10,7,Rain-Snow,62
2014,12,X3,51,44,37,49,42,24,100,⋯,10,5,1,29,12,38,0.44,8,Rain,254
2014,12,X4,43,37,30,24,21,13,69,⋯,10,10,10,25,12,33,0.00,3,,292
2014,12,X5,42,34,26,37,25,12,85,⋯,10,10,5,22,10,26,0.11,5,Rain,61
2014,12,X6,45,42,38,45,40,36,100,⋯,10,4,0,22,8,25,1.09,8,Rain,313


In [5]:
tidy_weather %>% glimpse

Rows: 8,866
Columns: 26
$ year                      <int> 2014, 2014, 2014, 2014, 2014, 2014, 2014, 20…
$ month                     <int> 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, …
$ day                       <chr> "X1", "X2", "X3", "X4", "X5", "X6", "X7", "X…
$ X                         <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ Max.TemperatureF          <chr> "64", "42", "51", "43", "42", "45", "38", "2…
$ Mean.TemperatureF         <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ Min.TemperatureF          <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ Max.Dew.PointF            <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ MeanDew.PointF            <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ Min.DewpointF             <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ Max.Humidity              <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ Mean.Humidity             <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ Min.Humidity  

A few things about this data set are still odd. For one, the names of days start with an `X`. We can fix this with the `str_replace` function. We saw this a few labs back, but let's review!

In [ ]:

# str_replace replaces only the first instance of a substring (2nd arg)
str_replace("tattoo", "t", "l")

str_replace("tattoo", "tatt", "yah")

# use replace all to replace multiple occurrences
str_replace_all("tattoo", "t", "b")

# Replace $ with nothing, so it removes the dollar sign
# we are applying this function to a vector!
# notice the use of \\...
# this is because $ is a reserved regex character
cost = c("$8", "12.5$", "$45")
cost = str_replace_all(cost, "\\$", "")
print(cost) 

#change its type to numeric
cost = as.numeric(cost)
print(cost)

#### Exercise 1: Remove `X` from the `day` entries and change its type to `numeric`

In [17]:
tidy_weather <- tidy_weather %>% mutate(day = as.numeric(str_replace(day, "X", "")))
tidy_weather %>% head

year,month,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,⋯,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2014,12,1,64,52,39,46,40,26,74,⋯,10,10,10,22,13,29,0.01,6,Rain,268
2014,12,2,42,38,33,40,27,17,92,⋯,10,8,2,24,15,29,0.10,7,Rain-Snow,62
2014,12,3,51,44,37,49,42,24,100,⋯,10,5,1,29,12,38,0.44,8,Rain,254
2014,12,4,43,37,30,24,21,13,69,⋯,10,10,10,25,12,33,0.00,3,,292
2014,12,5,42,34,26,37,25,12,85,⋯,10,10,5,22,10,26,0.11,5,Rain,61
2014,12,6,45,42,38,45,40,36,100,⋯,10,4,0,22,8,25,1.09,8,Rain,313


#### Exercise 2: Combine the year, month, and day columns into a new column called `date`.

*Hint: Use the `unite` function.*

In [18]:
tidy_weather <- tidy_weather %>% unite(col=date, year, month, day, remove=FALSE, sep= ".")
tidy_weather %>% head

date,year,month,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,⋯,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2014.12.1,2014,12,1,64,52,39,46,40,26,⋯,10,10,10,22,13,29,0.01,6,Rain,268
2014.12.2,2014,12,2,42,38,33,40,27,17,⋯,10,8,2,24,15,29,0.10,7,Rain-Snow,62
2014.12.3,2014,12,3,51,44,37,49,42,24,⋯,10,5,1,29,12,38,0.44,8,Rain,254
2014.12.4,2014,12,4,43,37,30,24,21,13,⋯,10,10,10,25,12,33,0.00,3,,292
2014.12.5,2014,12,5,42,34,26,37,25,12,⋯,10,10,5,22,10,26,0.11,5,Rain,61
2014.12.6,2014,12,6,45,42,38,45,40,36,⋯,10,4,0,22,8,25,1.09,8,Rain,313


#### Exercise 3: Move events variable to the second column (Just after the `date`)

In [19]:
tidy_weather <- tidy_weather %>% 
  select(date, Events, everything())
tidy_weather %>% head

date,Events,year,month,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,⋯,Min.Sea.Level.PressureIn,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,WindDirDegrees
<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2014.12.1,Rain,2014,12,1,64,52,39,46,40,⋯,30.01,10,10,10,22,13,29,0.01,6,268
2014.12.2,Rain-Snow,2014,12,2,42,38,33,40,27,⋯,30.4,10,8,2,24,15,29,0.10,7,62
2014.12.3,Rain,2014,12,3,51,44,37,49,42,⋯,29.87,10,5,1,29,12,38,0.44,8,254
2014.12.4,,2014,12,4,43,37,30,24,21,⋯,30.09,10,10,10,25,12,33,0.00,3,292
2014.12.5,Rain,2014,12,5,42,34,26,37,25,⋯,30.45,10,10,5,22,10,26,0.11,5,61
2014.12.6,Rain,2014,12,6,45,42,38,45,40,⋯,30.16,10,4,0,22,8,25,1.09,8,313


#### Exercise 4: `PrecipitationIn` has “T”s for "Trace." “Traces” are defined as precipitation of less than 0.005 inch. So, in this case, we need to map “T”s to 0.

In [21]:
tidy_weather <- tidy_weather %>% 
  mutate(PrecipitationIn = ifelse(PrecipitationIn == "T", "0", PrecipitationIn))

### Fun exercise: What is happening cell in below?

In [22]:
l = list(as.numeric, sqrt, `+`, c('1','9'))

l[[2]](l[[3]](l[[1]](l[[4]])[1], l[[1]](l[[4]])[2]))

[1] 3.162278

We are casting the strings '1' and '9' to integers, summing them, and taking the square root. Thus the answer is root 10.

#### Exercise 5: What are the unique events in the dataset?

In [34]:
tidy_weather %>% select(Events) %>% unique %>% deframe

[1] "Rain"                       "Rain-Snow"                 
 [3] ""                           "Snow"                      
 [5] "Fog-Rain"                   "Fog-Snow"                  
 [7] "Fog-Rain-Snow"              NA                          
 [9] "Fog-Rain-Thunderstorm"      "Fog"                       
[11] "Rain-Thunderstorm"          "Thunderstorm"              
[13] "Fog-Rain-Hail-Thunderstorm"

#### Exercise 6: An empty entry means that there is no weather event. Change empties to `Clear`.

In [35]:
tidy_weather <- tidy_weather %>% 
  mutate(Events = ifelse(Events == "", "Clear", Events))

tidy_weather %>% select(Events) %>% unique %>% deframe

[1] "Rain"                       "Rain-Snow"                 
 [3] "Clear"                      "Snow"                      
 [5] "Fog-Rain"                   "Fog-Snow"                  
 [7] "Fog-Rain-Snow"              NA                          
 [9] "Fog-Rain-Thunderstorm"      "Fog"                       
[11] "Rain-Thunderstorm"          "Thunderstorm"              
[13] "Fog-Rain-Hail-Thunderstorm"

In [37]:
tidy_weather %>% head

date,Events,year,month,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,⋯,Min.Sea.Level.PressureIn,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,WindDirDegrees
<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2014.12.1,Rain,2014,12,1,64,52,39,46,40,⋯,30.01,10,10,10,22,13,29,0.01,6,268
2014.12.2,Rain-Snow,2014,12,2,42,38,33,40,27,⋯,30.4,10,8,2,24,15,29,0.10,7,62
2014.12.3,Rain,2014,12,3,51,44,37,49,42,⋯,29.87,10,5,1,29,12,38,0.44,8,254
2014.12.4,Clear,2014,12,4,43,37,30,24,21,⋯,30.09,10,10,10,25,12,33,0.00,3,292
2014.12.5,Rain,2014,12,5,42,34,26,37,25,⋯,30.45,10,10,5,22,10,26,0.11,5,61
2014.12.6,Rain,2014,12,6,45,42,38,45,40,⋯,30.16,10,4,0,22,8,25,1.09,8,313


#### Addendum

Most measurement columns are of string type. Here's how we can convert them to numeric type.

In [38]:
tidy_weather = tidy_weather %>% 
  mutate_each(funs = list(as.numeric), Max.TemperatureF:WindDirDegrees)
tidy_weather %>% head

Warning message:
“`mutate_each_()` was deprecated in dplyr 0.7.0.
Please use `across()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was generated.”


date,Events,year,month,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,⋯,Min.Sea.Level.PressureIn,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,WindDirDegrees
<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2014.12.1,Rain,2014,12,1,64,52,39,46,40,⋯,30.01,10,10,10,22,13,29,0.01,6,268
2014.12.2,Rain-Snow,2014,12,2,42,38,33,40,27,⋯,30.40,10,8,2,24,15,29,0.10,7,62
2014.12.3,Rain,2014,12,3,51,44,37,49,42,⋯,29.87,10,5,1,29,12,38,0.44,8,254
2014.12.4,Clear,2014,12,4,43,37,30,24,21,⋯,30.09,10,10,10,25,12,33,0.00,3,292
2014.12.5,Rain,2014,12,5,42,34,26,37,25,⋯,30.45,10,10,5,22,10,26,0.11,5,61
2014.12.6,Rain,2014,12,6,45,42,38,45,40,⋯,30.16,10,4,0,22,8,25,1.09,8,313


Reference: [Cleaning Messy Weather Dataset with tidyverse](https://www.rpubs.com/justinhtet/cleaning-messy-weather-dataset-with-tidyverse)